# Introduction

This notebook contains a tutorial on how to use transition matrices of exogenous processes in respy. The corresponding function in respy is `parse_transition_matrix_for_exogenous_processes`. It translates a transition matrix to the specification files `params`and `options` used in respy.

In [3]:
from respy import parse_transition_matrix_for_exogenous_processes
import pandas as pd

This notebook contains three showcases. A exogenous process wit probabilities depending *only* on the current state of the process, a process depending on a combination of general state variables and the current state of the process and one process only depending on general state variables and not the current state of the process.

The case of an exogenous process with constants probabilities across states is quite easy and explained in the general tutorial on exogenous processes. The distinction of this tutorial is the use of transition matrices. A general specification of also complex exogenous processes depending on logit coefficients instead of probabilities are also included in the general tutorial on exogenous processes.

To fully understand this tutorial it is advised to first study the general tutorial on exogenous processes.

# Transition matrices

In a first step, the three transition matrices are specified. Throughout this tutorial, the exogenous process used for demonstration, will be th one of a `health_shock`. We assume two states of the process: `healthy` and `sick`.

In [4]:
process_name = "health_shock"
process_states = ["healthy", "sick"]

Now the three transition matrices are defined:

In [2]:
# 
df = pd.DataFrame(
    columns=["sick", "healthy"],
    index=["old", "young"],
    data=[[0.8, 0.2],
         [0.6, 0.4]],
)

NameError: name 'pd' is not defined

In [4]:
df

,sick,healthy
old,0.8,0.2
young,0.6,0.4


In [5]:
def parse_exog_process_matrix(matrix, process_name):
    states, process_states = check_states_size(matrix)
    covariates = create_options_entries(states, process_name, process_states)
    transformed_matrix = _transform_matrix(matrix)
    params = _create_params(transformed_matrix, states, process_name, process_states)
    return params, covariates


def _create_params(transformed_matrix, states, process_name, process_states):
    categories = [
        f"exogenous_process_{process_name}_{state}" for state in process_states
    ]
    index = pd.MultiIndex.from_tuples(
        itertools.product(categories, states), names=["category", "name"]
    )
    params = pd.DataFrame(index=index, columns=["value"])
    for process_state in process_states:
        params.loc[
            (f"exogenous_process_health_shock_{process_state}", states), "value"
        ] = transformed_matrix[process_state].values
    return params

    
def _transform_matrix(matrix):
    transformed_matrix = matrix.copy()
    transformed_matrix[transformed_matrix == 0] = -1e300
    transformed_matrix[transformed_matrix > 0] = np.log(
        transformed_matrix[transformed_matrix > 0]
    )
    transformed_matrix.index = transformed_matrix.index.map(str)
    return transformed_matrix


def create_options_entries(states, process_name, process_states):
    covariates = {}
    for state in states:
        i = 0
        for process_state in process_states:
            if process_state in state:
                destination_state = process_state
                i += 1
        if i == 0:
            covariates[str(state)] = "?"
        elif i == 1:
            covariates[str(state)] = f"{process_name} == {destination_state} & ?"
        else:
            raise ValueError(f"{state} contains more than one process state.")
    return covariates


def check_states_size(matrix):

    return states, process_states


def _check_numerics(matrix, n_states):
    assert (matrix.sum(axis=1) == np.full(n_states, 1)).all()
    assert (((matrix >= 0) & (matrix <= 1)).all()).all()